In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import optuna
import  os
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from utils import Optuna_for_LGB
from preprocess import convert_notation, region_encoding, region_grouping, correlation_to_pca
PATH = '../data/'
pd.set_option('display.max_columns', 100)

In [8]:
train_df = pd.read_csv(PATH + 'train.csv')
test_df = pd.read_csv(PATH + 'test.csv')
train_df = convert_notation(train_df)
test_df = convert_notation(test_df)
train_df, test_df, le = region_encoding(train_df, test_df)
train_df, test_df = correlation_to_pca(train_df, test_df)
columns = list(test_df.columns)
columns.remove('index')
columns.remove('region')
train_df, test_df = region_grouping(train_df, test_df, columns, diff=True)
print(train_df.shape, test_df.shape)
train_df.head()

(4043, 88) (4046, 87)


,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region,tempo_min,tempo_max,pca_loudness_acousticness,pca_energy_acousticness,pca_danceability_positiveness,pca_energy_loudness,popularity_mean,popularity_std,duration_ms_mean,duration_ms_std,acousticness_mean,acousticness_std,positiveness_mean,positiveness_std,danceability_mean,danceability_std,loudness_mean,loudness_std,energy_mean,energy_std,liveness_mean,liveness_std,speechiness_mean,speechiness_std,instrumentalness_mean,instrumentalness_std,tempo_mean,tempo_std,tempo_min_mean,tempo_min_std,tempo_max_mean,tempo_max_std,pca_loudness_acousticness_mean,pca_loudness_acousticness_std,pca_energy_acousticness_mean,pca_energy_acousticness_std,pca_danceability_positiveness_mean,pca_danceability_positiveness_std,pca_energy_loudness_mean,pca_energy_loudness_std,popularity_region_mean_diff,popularity_region_mean_diff_region_std_diff,duration_ms_region_mean_diff,duration_ms_region_mean_diff_region_std_diff,acousticness_region_mean_diff,acousticness_region_mean_diff_region_std_diff,positiveness_region_mean_diff,positiveness_region_mean_diff_region_std_diff,danceability_region_mean_diff,danceability_region_mean_diff_region_std_diff,loudness_region_mean_diff,loudness_region_mean_diff_region_std_diff,energy_region_mean_diff,energy_region_mean_diff_region_std_diff,liveness_region_mean_diff,liveness_region_mean_diff_region_std_diff,speechiness_region_mean_diff,speechiness_region_mean_diff_region_std_diff,instrumentalness_region_mean_diff,instrumentalness_region_mean_diff_region_std_diff,tempo_region_mean_diff,tempo_region_mean_diff_region_std_diff,tempo_min_region_mean_diff,tempo_min_region_mean_diff_region_std_diff,tempo_max_region_mean_diff,tempo_max_region_mean_diff_region_std_diff,pca_loudness_acousticness_region_mean_diff,pca_loudness_acousticness_region_mean_diff_region_std_diff,pca_energy_acousticness_region_mean_diff,pca_energy_acousticness_region_mean_diff_region_std_diff,pca_danceability_positiveness_region_mean_diff,pca_danceability_positiveness_region_mean_diff_region_std_diff,pca_energy_loudness_region_mean_diff,pca_energy_loudness_region_mean_diff_region_std_diff
0,0,10,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,31,7,121,152,-5.794789,-0.356869,-0.422632,-5.796926,33.594805,16.577375,241215.114286,74969.668951,0.361314,0.244860,0.417410,0.222599,0.464421,0.173076,-8.325096,4.257331,0.589758,0.223368,0.273181,0.167866,0.199170,0.083200,0.234180,0.172665,24.574026,5.750032,109.535065,27.384025,134.109091,30.246187,0.650025,4.259049,0.026307,0.304540,-0.060621,0.250798,0.649891,4.260402,-22.594805,-39.172180,-40121.114286,-115090.783237,-0.248502,-0.493362,-0.260163,-0.482762,-0.276579,-0.449655,6.440244,2.182912,0.304160,0.080792,0.090387,-0.077479,0.190938,0.107738,0.654704,0.482039,6.425974,0.675942,11.464935,-15.919090,17.890909,-12.355278,-6.444815,-10.703863,-0.383175,-0.687715,-0.362011,-0.612809,-6.446817,-10.707219
1,1,8,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,23,8,153,176,-2.135171,-0.353878,-0.078569,-2.136684,40.509218,16.328317,239207.812460,75697.170685,0.318765,0.227062,0.486135,0.214958,0.508781,0.157784,-7.385909,3.817392,0.621247,0.190882,0.273330,0.158975,0.205838,0.088623,0.199632,0.128815,24.192626,5.735288,110.173554,28.945391,134.366179,31.016197,-0.290019,3.818753,-0.026607,0.271029,0.020990,0.237063,-0.289823,3.819206,28.490782,12.162465,69285.187540,-6411.983146,-0.217432,-0.444494,-0.139572,-0.354530,0.045663,-0.112121,1.839414,-1.977977,0.253162,0.062280,-0.079439,-0.238413,-0.044342,-0.132965,-0.075721,-0.204536,-1.192626,-6.927914,42.826446,13.881056,41.633821,10.617623,-1.845153,-5.663906,-0.327271,-0.598300,-0.099559,-0.336622,-1.846860,-5.666066
2,2,3,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,12,4,64,76,1.584262,0.225051,-0.196643,1.584960,43.98

In [9]:
test_df.head()

,index,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region,tempo_min,tempo_max,pca_loudness_acousticness,pca_energy_acousticness,pca_danceability_positiveness,pca_energy_loudness,popularity_mean,popularity_std,duration_ms_mean,duration_ms_std,acousticness_mean,acousticness_std,positiveness_mean,positiveness_std,danceability_mean,danceability_std,loudness_mean,loudness_std,energy_mean,energy_std,liveness_mean,liveness_std,speechiness_mean,speechiness_std,instrumentalness_mean,instrumentalness_std,tempo_mean,tempo_std,tempo_min_mean,tempo_min_std,tempo_max_mean,tempo_max_std,pca_loudness_acousticness_mean,pca_loudness_acousticness_std,pca_energy_acousticness_mean,pca_energy_acousticness_std,pca_danceability_positiveness_mean,pca_danceability_positiveness_std,pca_energy_loudness_mean,pca_energy_loudness_std,popularity_region_mean_diff,popularity_region_mean_diff_region_std_diff,duration_ms_region_mean_diff,duration_ms_region_mean_diff_region_std_diff,acousticness_region_mean_diff,acousticness_region_mean_diff_region_std_diff,positiveness_region_mean_diff,positiveness_region_mean_diff_region_std_diff,danceability_region_mean_diff,danceability_region_mean_diff_region_std_diff,loudness_region_mean_diff,loudness_region_mean_diff_region_std_diff,energy_region_mean_diff,energy_region_mean_diff_region_std_diff,liveness_region_mean_diff,liveness_region_mean_diff_region_std_diff,speechiness_region_mean_diff,speechiness_region_mean_diff_region_std_diff,instrumentalness_region_mean_diff,instrumentalness_region_mean_diff_region_std_diff,tempo_region_mean_diff,tempo_region_mean_diff_region_std_diff,tempo_min_region_mean_diff,tempo_min_region_mean_diff_region_std_diff,tempo_max_region_mean_diff,tempo_max_region_mean_diff_region_std_diff,pca_loudness_acousticness_region_mean_diff,pca_loudness_acousticness_region_mean_diff_region_std_diff,pca_energy_acousticness_region_mean_diff,pca_energy_acousticness_region_mean_diff_region_std_diff,pca_danceability_positiveness_region_mean_diff,pca_danceability_positiveness_region_mean_diff_region_std_diff,pca_energy_loudness_region_mean_diff,pca_energy_loudness_region_mean_diff_region_std_diff
0,4046,43,192465,0.559812,0.565035,0.522237,-12.217087,0.620510,0.389727,0.214989,0.161924,19,8,77,96,4.546242,0.163635,0.096455,4.538670,40.509218,16.328317,239207.812460,75697.170685,0.318765,0.227062,0.486135,0.214958,0.508781,0.157784,-7.385909,3.817392,0.621247,0.190882,0.273330,0.158975,0.205838,0.088623,0.199632,0.128815,24.192626,5.735288,110.173554,28.945391,134.366179,31.016197,-0.290019,3.818753,-0.026607,0.271029,0.020990,0.237063,-0.289823,3.819206,2.490782,-13.837535,-46742.812460,-122439.983146,0.241047,0.013984,0.078900,-0.136058,0.013456,-0.144328,-4.831178,-8.648569,-0.000737,-0.191620,0.116397,-0.042578,0.009151,-0.079472,-0.037708,-0.166523,-5.192626,-10.927914,-33.173554,-62.118944,-38.366179,-69.382377,4.836260,1.017508,0.190242,-0.080787,0.075464,-0.161598,4.828493,1.009288
1,4047,40,226536,0.134002,0.542547,0.493128,-2.612846,0.853089,0.163106,0.184183,0.155460,31,19,121,152,-5.066487,-0.315013,0.062626,-5.067973,41.011494,16.476096,244900.321839,93232.211281,0.343493,0.240269,0.459062,0.220446,0.498184,0.156866,-7.753427,4.007073,0.610866,0.200528,0.267327,0.158768,0.204632,0.086357,0.212711,0.150883,24.619253,5.436633,110.886494,27.971878,135.505747,29.839040,0.078079,4.009015,-0.000737,0.290748,-0.007797,0.242133,0.077836,4.009356,-1.011494,-17.487590,-18364.321839,-111596.533120,-0.209490,-0.449760,0.083485,-0.136961,-0.005056,-0.161922,5.140581,1.133508,0.242224,0.041696,-0.104221,-0.262989,-0.020449,-0.106806,-0.057252,-0.208134,6.380747,0.944114,10.113506,-17.858372,16.494253,-13.344787,-5.144567,-9.153582,-0.314275,-0.605023,0.070423,-0.171710,-5.145809,-9.155166
2,4048,58,215385,0.140537,0.643086,0.533189,-3.439684,0.834416,0.429304,0.167352,0.179191,31,10,121,152,-4.239831,-0.298355,0.169936,-4.240974,40.45021

In [11]:
train_df[test_df.columns].describe()

,index,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region,tempo_min,tempo_max,pca_loudness_acousticness,pca_energy_acousticness,pca_danceability_positiveness,pca_energy_loudness,popularity_mean,popularity_std,duration_ms_mean,duration_ms_std,acousticness_mean,acousticness_std,positiveness_mean,positiveness_std,danceability_mean,danceability_std,loudness_mean,loudness_std,energy_mean,energy_std,liveness_mean,liveness_std,speechiness_mean,speechiness_std,instrumentalness_mean,instrumentalness_std,tempo_mean,tempo_std,tempo_min_mean,tempo_min_std,tempo_max_mean,tempo_max_std,pca_loudness_acousticness_mean,pca_loudness_acousticness_std,pca_energy_acousticness_mean,pca_energy_acousticness_std,pca_danceability_positiveness_mean,pca_danceability_positiveness_std,pca_energy_loudness_mean,pca_energy_loudness_std,popularity_region_mean_diff,popularity_region_mean_diff_region_std_diff,duration_ms_region_mean_diff,duration_ms_region_mean_diff_region_std_diff,acousticness_region_mean_diff,acousticness_region_mean_diff_region_std_diff,positiveness_region_mean_diff,positiveness_region_mean_diff_region_std_diff,danceability_region_mean_diff,danceability_region_mean_diff_region_std_diff,loudness_region_mean_diff,loudness_region_mean_diff_region_std_diff,energy_region_mean_diff,energy_region_mean_diff_region_std_diff,liveness_region_mean_diff,liveness_region_mean_diff_region_std_diff,speechiness_region_mean_diff,speechiness_region_mean_diff_region_std_diff,instrumentalness_region_mean_diff,instrumentalness_region_mean_diff_region_std_diff,tempo_region_mean_diff,tempo_region_mean_diff_region_std_diff,tempo_min_region_mean_diff,tempo_min_region_mean_diff_region_std_diff,tempo_max_region_mean_diff,tempo_max_region_mean_diff_region_std_diff,pca_loudness_acousticness_region_mean_diff,pca_loudness_acousticness_region_mean_diff_region_std_diff,pca_energy_acousticness_region_mean_diff,pca_energy_acousticness_region_mean_diff_region_std_diff,pca_danceability_positiveness_region_mean_diff,pca_danceability_positiveness_region_mean_diff_region_std_diff,pca_energy_loudness_region_mean_diff,pca_energy_loudness_region_mean_diff_region_std_diff
count,4043.000000,4043.000000,4.043000e+03,4043.000000,4033.000000,4035.000000,4043.000000,4043.000000,4040.000000,4035.000000,4042.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4.043000e+03,4.043000e+03,4043.000000,4043.000000,4033.000000,4033.000000,4035.000000,4035.000000,4043.000000,4043.000000,4043.000000,4043.000000,4040.000000,4040.000000,4035.000000,4035.000000,4042.000000,4042.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000,4043.000000
mean,2022.611674,41.061835,2.421725e+05,0.346451,0.464164,0.504457,-7.714284,0.603751,0.265991,0.198704,0.214356,24.392530,8.816473,110.154341,134.546871,0.039044,0.005978,-0.000308,0.038953,41.102956,15.658754,242250.259574,78798.185763,0.340290,0.234230,0.465268,0.219517,0.501686,0.157179,-7.669050,3.975724,0.606588,0.196781,0.267608,0.154361,0.200281,0.081830,0.215764,0.146264,24.303739,5.636406,110.026280,28.623198,134.330019,30.620456,-0.006357,3.977446,-0.000633,0.282369,-0.000700,0.241788,-0.006351,3.977881,-0.041121,-15.699875,-7.779853e+01,-7.887598e+04,0.006160,-0.228069,-0.001124,-0.220606,0.002768,-0.154387,-0.045234,-4.020959,-0.002837,-0.199618,-0.001619,-0.155978,-0.001538,-0.083344,-0.001397,-0.

In [12]:
test_df.describe()

,index,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region,tempo_min,tempo_max,pca_loudness_acousticness,pca_energy_acousticness,pca_danceability_positiveness,pca_energy_loudness,popularity_mean,popularity_std,duration_ms_mean,duration_ms_std,acousticness_mean,acousticness_std,positiveness_mean,positiveness_std,danceability_mean,danceability_std,loudness_mean,loudness_std,energy_mean,energy_std,liveness_mean,liveness_std,speechiness_mean,speechiness_std,instrumentalness_mean,instrumentalness_std,tempo_mean,tempo_std,tempo_min_mean,tempo_min_std,tempo_max_mean,tempo_max_std,pca_loudness_acousticness_mean,pca_loudness_acousticness_std,pca_energy_acousticness_mean,pca_energy_acousticness_std,pca_danceability_positiveness_mean,pca_danceability_positiveness_std,pca_energy_loudness_mean,pca_energy_loudness_std,popularity_region_mean_diff,popularity_region_mean_diff_region_std_diff,duration_ms_region_mean_diff,duration_ms_region_mean_diff_region_std_diff,acousticness_region_mean_diff,acousticness_region_mean_diff_region_std_diff,positiveness_region_mean_diff,positiveness_region_mean_diff_region_std_diff,danceability_region_mean_diff,danceability_region_mean_diff_region_std_diff,loudness_region_mean_diff,loudness_region_mean_diff_region_std_diff,energy_region_mean_diff,energy_region_mean_diff_region_std_diff,liveness_region_mean_diff,liveness_region_mean_diff_region_std_diff,speechiness_region_mean_diff,speechiness_region_mean_diff_region_std_diff,instrumentalness_region_mean_diff,instrumentalness_region_mean_diff_region_std_diff,tempo_region_mean_diff,tempo_region_mean_diff_region_std_diff,tempo_min_region_mean_diff,tempo_min_region_mean_diff_region_std_diff,tempo_max_region_mean_diff,tempo_max_region_mean_diff_region_std_diff,pca_loudness_acousticness_region_mean_diff,pca_loudness_acousticness_region_mean_diff_region_std_diff,pca_energy_acousticness_region_mean_diff,pca_energy_acousticness_region_mean_diff_region_std_diff,pca_danceability_positiveness_region_mean_diff,pca_danceability_positiveness_region_mean_diff_region_std_diff,pca_energy_loudness_region_mean_diff,pca_energy_loudness_region_mean_diff_region_std_diff
count,4046.000000,4046.000000,4.046000e+03,4045.000000,4032.000000,4035.000000,4046.000000,4045.000000,4040.000000,4035.000000,4044.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4045.000000,4045.000000,4032.000000,4032.000000,4035.000000,4035.000000,4046.000000,4046.000000,4045.000000,4045.000000,4040.000000,4040.000000,4035.000000,4035.000000,4044.000000,4044.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000,4046.000000
mean,6068.500000,41.158181,2.418486e+05,0.335216,0.467855,0.499143,-7.636531,0.608755,0.268926,0.201776,0.216227,24.207612,8.673752,109.867029,134.074642,-0.039015,-0.005974,0.000307,-0.038925,41.117090,15.672501,241770.850604,78691.484322,0.341372,0.234608,0.466737,0.219629,0.501927,0.157273,-7.681731,3.972494,0.605917,0.196834,0.267313,0.153946,0.200381,0.082008,0.214854,0.145161,24.296338,5.645183,109.994995,28.619990,134.291333,30.641010,0.006352,3.974209,0.000633,0.282675,0.000699,0.241951,0.006346,3.974654,0.041091,-15.631410,77.740844,-78613.743478,-0.006157,-0.240759,0.001125,-0.218455,-0.002768,-0.159991,0.045201,-3.927294,0.002836,-0.193992,0.001619,-0.152320,0.001538,-0.080385,0.001396,-0.143730,-0.088

In [16]:
(train_df[test_df.columns].describe() - test_df.describe()) / pd.concat([train_df[test_df.columns], test_df]).describe().iloc[6]

,index,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region,tempo_min,tempo_max,pca_loudness_acousticness,pca_energy_acousticness,pca_danceability_positiveness,pca_energy_loudness,popularity_mean,popularity_std,duration_ms_mean,duration_ms_std,acousticness_mean,acousticness_std,positiveness_mean,positiveness_std,danceability_mean,danceability_std,loudness_mean,loudness_std,energy_mean,energy_std,liveness_mean,liveness_std,speechiness_mean,speechiness_std,instrumentalness_mean,instrumentalness_std,tempo_mean,tempo_std,tempo_min_mean,tempo_min_std,tempo_max_mean,tempo_max_std,pca_loudness_acousticness_mean,pca_loudness_acousticness_std,pca_energy_acousticness_mean,pca_energy_acousticness_std,pca_danceability_positiveness_mean,pca_danceability_positiveness_std,pca_energy_loudness_mean,pca_energy_loudness_std,popularity_region_mean_diff,popularity_region_mean_diff_region_std_diff,duration_ms_region_mean_diff,duration_ms_region_mean_diff_region_std_diff,acousticness_region_mean_diff,acousticness_region_mean_diff_region_std_diff,positiveness_region_mean_diff,positiveness_region_mean_diff_region_std_diff,danceability_region_mean_diff,danceability_region_mean_diff_region_std_diff,loudness_region_mean_diff,loudness_region_mean_diff_region_std_diff,energy_region_mean_diff,energy_region_mean_diff_region_std_diff,liveness_region_mean_diff,liveness_region_mean_diff_region_std_diff,speechiness_region_mean_diff,speechiness_region_mean_diff_region_std_diff,instrumentalness_region_mean_diff,instrumentalness_region_mean_diff_region_std_diff,tempo_region_mean_diff,tempo_region_mean_diff_region_std_diff,tempo_min_region_mean_diff,tempo_min_region_mean_diff_region_std_diff,tempo_max_region_mean_diff,tempo_max_region_mean_diff_region_std_diff,pca_loudness_acousticness_region_mean_diff,pca_loudness_acousticness_region_mean_diff_region_std_diff,pca_energy_acousticness_region_mean_diff,pca_energy_acousticness_region_mean_diff_region_std_diff,pca_danceability_positiveness_region_mean_diff,pca_danceability_positiveness_region_mean_diff_region_std_diff,pca_energy_loudness_region_mean_diff,pca_energy_loudness_region_mean_diff_region_std_diff
count,-0.000494,-0.057692,-0.000011,-3.932550,1.555002,0.000000,0.611411,-2.594724,0.000000,0.000000,-9.730707,-0.096774,-0.230769,-0.024793,-0.019737,-1.432236,-15.580619,-15.410922,-1.431372,-0.070155,-0.182082,-0.000012,-0.000036,-8.605804,-12.148029,-6.171129,-13.477170,-5.795770,-18.599507,0.406179,-0.715481,-4.828999,-14.550070,-10.975729,-18.804166,-14.574552,-33.851142,-13.814010,-18.199975,-0.122080,-0.521736,-0.027055,-0.103158,-0.022139,-0.096724,-5.375425,-0.715179,-238.114272,-9.981930,-142.923836,-12.167384,-5.370962,-0.715106,-0.285965,0.620151,-0.000099,0.000070,-12.454907,26.896884,5.648400,-23.023150,0.000000,-0.000000,-1.111486,2.445899,-12.581847,54.333074,0.000000,-0.000000,0.000000,-0.000000,-1597.661899,16.503135,-0.466855,-3.344136,-0.244503,0.189414,-0.158451,0.248548,-1.473660,1.594231,-16.388923,30.286374,-15.668614,58.684403,-1.475376,1.594840
mean,-0.666648,-0.001853,0.001189,0.022091,-0.005739,0.008611,0.015847,-0.006492,-0.009133,-0.013534,-0.009106,0.005965,0.010979,0.002374,0.003107,0.037266,0.062073,-0.003160,0.037157,-0.000331,-0.000834,0.001935,0.001290,-0.003103,-0.001534,-0.003022,-0.000504,-0.000465,-0.000587,-0.001717,0.000770,0.001080,-0.000256,0.001079,0.002602,-0.000487,-0.002008,0.004189,0.006691,0.000301,-0.001526,0.000282,0.000110,0.000285,-0.000663,-0.022771,0.000772,-0.100508,-0.001018,-0.066664,-0.000663,-0.022731,0.000769,-0.007837,0.014153,-0.005145,0.006137,0.076709,-0.170663,-0.012703,0.049519,0.049284,-0.128446,-0.033506,0.076365,-0.035686,0.152851,-0.060596,0.037928,-0.103846,0.061085,-2.231105,0.032292,0.027625,0.207663,0.020866,-0.015963,0.022898,-0.037622,0.044587,-0.046515,0.072211,-0.136533,0.004096,-0.018538,0.044544,-0.046435
std,-0.000017,0.001210,0.025410,0.009450,0.007001,-0.0048

In [17]:
pd.plotting.scatter_matrix(train_df[test_df.columns])

AttributeError: 'PlotAccessor' object has no attribute 'scatter_matrix'

other

In [5]:
genre_df = pd.read_csv(PATH + 'genre_labels.csv')
genre_df

,genre,labels
0,country,0
1,electronic,1
2,folk,2
3,hip-hop,3
4,jazz,4
5,latin,5
6,classic,6
7,other-light-music,7
8,pop,8
9,religious,9


In [6]:
sub_df = pd.read_csv(PATH + 'sample_submit.csv', header=None, names=['ID', 'Pred'])
sub_df.head()

,ID,Pred
0,4046,9
1,4047,8
2,4048,4
3,4049,2
4,4050,5
